<a href="https://colab.research.google.com/github/ManG0A2/MLP-class/blob/main/4_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#앙상블 학습
#정형 데이터를 다루는데 가장 뛰어난 알고리즘
#결정 트리를 기반

In [2]:
#랜덤 포레스트
#결정트리는 각 특성마다 하나가 남을때까지 분류하기에 과대적합 될 가능성이 높음
#그래서 특성을 제한하는 방식을 사용

#1. 훈련하기 위한 데이터를 랜덤하게 만든다.
#-랜덤하게 추출한 부트스트랩 샘플을 만듬(중복된 샘플 추출 가능)
# 하나 뽑고 다시 넣고 다시 뽑고 반복
#부트 스트랩 샘플은 주로 훈련 세트와 같은 크기로 만든다 25%정도

#2. 부트스트랩 샘플로 결정 트리를 훈련
#-특성 집합의 루트 갯수만큼 램덤 선택해 그중에서 또 최선의 분할을 선택
#-위는 분류만 해당되고 회귀의 경우는 모든 특성집합을 사용한다.

#3. 정해진 수만큼 위 과정을 반복

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split( data, target, test_size = 0.2, random_state=42,stratify=target)

In [4]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)

scores = cross_validate( rf, train_input, train_target,
                       return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9981720130385032 0.8937817428000298


In [5]:
#특성 중요도
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.2301315  0.50081183 0.26905668]


In [6]:
'''
이 결과를 앞의 1절 '결정 트리'에서 만든 특성 중요도와 비교해보자. 결정트리에서 특성 중요도는 다음과 같았다.

0.12345626 , 0.86862934 , 0.0079144

각각 [알코올 도수, 당도, PH]였는데, 두 번째 특성인 당도의 중요도가 감소하고 알코올 도수와 PH특성의 중요도가 조금 상승했다.
이런 이유는 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문이다. 그 결과 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻는다. 이는 과대적합을 줄이고 일반화 성능을 높이는데 도움이 된다.

또한 두번째로 랜덤 포레스트의 신기한 특징은 부트스트랩 샘플에 포함되지 않고 남는 샘플이 있다.
이런 샘프을 OOB (out of bag) 샘플이라고 한다. 이 남는 샘플을 사용하여 부트스트랩 샘플로 훈련한 결정 트리를 평가할 수 있다. 마치 검증 세트의 역할을 하는 것이다.
이 점수를 얻으려면 RandomForestClassifier 클래스의 oob_score 매개변수를 True로 지정해야 한다.
'''

"\n이 결과를 앞의 1절 '결정 트리'에서 만든 특성 중요도와 비교해보자. 결정트리에서 특성 중요도는 다음과 같았다.\n\n0.12345626 , 0.86862934 , 0.0079144\n\n각각 [알코올 도수, 당도, PH]였는데, 두 번째 특성인 당도의 중요도가 감소하고 알코올 도수와 PH특성의 중요도가 조금 상승했다. \n이런 이유는 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문이다. 그 결과 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻는다. 이는 과대적합을 줄이고 일반화 성능을 높이는데 도움이 된다.\n\n또한 두번째로 랜덤 포레스트의 신기한 특징은 부트스트랩 샘플에 포함되지 않고 남는 샘플이 있다. \n이런 샘프을 OOB (out of bag) 샘플이라고 한다. 이 남는 샘플을 사용하여 부트스트랩 샘플로 훈련한 결정 트리를 평가할 수 있다. 마치 검증 세트의 역할을 하는 것이다.\n이 점수를 얻으려면 RandomForestClassifier 클래스의 oob_score 매개변수를 True로 지정해야 한다.\n"

In [7]:
#OOB 점수 출력
rf = RandomForestClassifier(oob_score=True, n_jobs= -1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8964787377333077


In [8]:
#2. 엑스트라 트리
#랜덤 포레스트랑 비슷하지만 노드를 분할 할 때 최선의 분할을 찾지않고 랜덤으로 분할함.
#또한 랜덤 포레스트와 엑스트라 트리의 차이점은 부트스트랩 샘플을 사용하지 않는다는 점이다.
#한 마디로 --> 각 결정 트리를 만들 때 전체 훈련 세트를 사용한다. 대신 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할 하자!

from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target,
                       return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9981720130385032 0.8924350336862368


In [9]:
#보통 엑스트라 트리가 무작위성이 좀 더 크기 때문에 랜덤 포레스트보다 더 많은 결정 트리를 훈련해야 한다. 하지만 랜덤하게 노드를 분할하기 때문에 빠른 계산 속도가 엑스트라 트리의 장점이다.
#아래는 엑스트라 트리에서 특성마다의 의존성
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20321321 0.51571223 0.28107456]


In [10]:
#그레이디언트 부스팅(SGD랑 비슷)
#깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법
#과대적합에 강하다 -> 일반화에 강하다.
#디폴트는 깊이가 3개짜리 트리가 100개

#이전모델을 점점 보와해가면서 순차적으로 더해나간 후 최종모델을 예측
#오차를 줄여나가는 과정

In [11]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target,
                       return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.886136193834053 0.8714622047827053


In [12]:
#학습률과 트리의 갯수 증가
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2,
                               random_state=42)

scores = cross_validate(gb, train_input, train_target,
                       return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9442947291395691 0.8751179018286814


In [13]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.17200523 0.67319793 0.15479683]


In [14]:
#히스토그램 기반 그레이디언트 부스팅

#히스토그램 기반 그레이디언트 부스팅은 먼저 입력 특성을 256개의 구간으로 나눈다. 따라서 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있다.
#예를 들어 키가 1~512까지 있다면 1,2 는 1 3,4 는 2로 ,, 몇이든 키의 구간을 256개로 나눈다.
#과대적합을 억제하면서 위보다 더 좋은 성능

In [15]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target,
                       return_train_score=True)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

/usr/local/lib/python3.10/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.930536541746549 0.8780021470348707


In [16]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target,
                                n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.09374639 0.23954204 0.08664614]


In [17]:
result = permutation_importance(hgb, test_input, test_target,
                               n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.04953846 0.20176923 0.04776923]


In [18]:
hgb.score(test_input, test_target)

0.8776923076923077

In [19]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target,
                       return_train_score=True)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9524726282724838 0.8785790701117939


In [20]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9524726282724838 0.8785790701117939


In [21]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.934577605325741 0.8805047382838529
